# Unified Brane-Cosmology — Inference Notebook v3 (Plug-and-Play)

**What this does**
- Loads PTA spectrum (if provided) and/or uses a synthetic demo.
- Loads a LISA sensitivity 'noise floor' (placeholder) if provided.
- Loads CMB/BBN priors on $\Delta N_{\rm eff}$ (Gaussian) and a BBN safety bound on $T_\times$.
- Links the break frequency via $f_{\rm br}(\lambda)\propto \lambda^{1/4}$ and maps $\mathcal C/\rho_{\gamma0}\leftrightarrow\Delta N_{\rm eff}$.

**How to use with real data**
1. Export PTA band points to `pta_spectrum.csv` with columns: `f_Hz, omega_gw, sigma`.
2. Export a LISA 'omega noise floor' to `lisa_sensitivity.csv` with columns: `f_Hz, omega_noise_floor`.
3. Provide `cmb_prior.json` with keys: `DeltaNeff_mean`, `DeltaNeff_sigma`.
4. Provide `bbn_constraint.json` with key: `Tx_min_MeV`.
5. Rerun the notebook; it will ingest your files and output posteriors and plots.


In [ ]:
import os, json, math, numpy as np, matplotlib.pyplot as plt
from brane_fit_lib import (load_pta_csv, load_lisa_csv, load_json,
                           f_break_of_lambda, DNeff_from_C_over,
                           C_over_rho_from_DNeff, omega_gw_broken,
                           like_pta_spectrum, like_lisa_upperlimit,
                           like_delta_neff, Tx_of_lambda)

# ---- File names (drop your real files next to this notebook) ----
pta_file = 'pta_spectrum.csv'
lisa_file = 'lisa_sensitivity.csv'
cmb_file = 'cmb_prior.json'
bbn_file = 'bbn_constraint.json'

# ---- Load data or fall back to synthetic demo ----
if os.path.exists(pta_file):
    f_pta, omg_pta, sig_pta = load_pta_csv(pta_file)
else:
    f_pta = np.array([1e-8, 2e-8, 5e-8])
    omg_pta = np.array([1.0e-9, 1.3e-9, 2.1e-9])
    sig_pta = np.array([5.0e-10, 6.0e-10, 7.5e-10])

if os.path.exists(lisa_file):
    f_lisa, omg_floor = load_lisa_csv(lisa_file)
else:
    f_lisa = np.array([1e-4, 3e-3, 1e-1])
    omg_floor = np.array([1e-12, 1e-12, 1e-11])

if os.path.exists(cmb_file):
    cmb = load_json(cmb_file)
    DNeff_mean, DNeff_sigma = cmb['DeltaNeff_mean'], cmb['DeltaNeff_sigma']
else:
    DNeff_mean, DNeff_sigma = 0.15, 0.10

if os.path.exists(bbn_file):
    bbn = load_json(bbn_file)
    Tx_min = bbn.get('Tx_min_MeV', 3.0)/1000.0  # GeV
else:
    Tx_min = 3.0/1000.0

# ---- Parameter grid ----
E_grid = np.logspace(-2, 3, 220)        # E_lambda in GeV
n1_grid = np.linspace(0.0, 3.5, 60)     # slopes can be refined later
n2_grid = np.linspace(-4.0, 1.0, 60)
Omega0_grid = np.logspace(-15, -7, 80)
DNeff_grid = np.linspace(-0.2, 0.6, 80)

# ---- Simple (factorized) scan to illustrate posteriors ----
best = {'LL':-1e99}
LL_E_D = np.zeros((E_grid.size, DNeff_grid.size))
for i,E in enumerate(E_grid):
    # BBN safety prior
    Tx = Tx_of_lambda(E)
    if Tx < Tx_min:  # hard cut; could be softened
        LL_E_D[i,:] = -1e9
        continue
    fbr = f_break_of_lambda(E)
    for j,DN in enumerate(DNeff_grid):
        # crude profile over (n1,n2,Omega0) by picking a quick match to PTA mid-band
        # This is a placeholder for a real optimizer or sampler.
        # Use a fixed pair of slopes for demo but try two normalizations to mimic profiling.
        n1, n2 = 2.0, -1.0
        # pick Omega0 to pass near the middle PTA point
        mid = len(f_pta)//2; target = omg_pta[mid]
        denom = (f_pta[mid]/fbr)**(n1 if f_pta[mid] < fbr else n2)
        Omega0_guess = max(1e-20, min(1e-6, target/denom))
        model_pta = omega_gw_broken(f_pta, fbr, n1, n2, Omega0_guess)
        LL_pta = -0.5*np.sum(((omg_pta - model_pta)/sig_pta)**2)
        # LISA penalty
        model_lisa = omega_gw_broken(f_lisa, fbr, n1, n2, Omega0_guess)
        ratio = model_lisa/np.maximum(omg_floor, 1e-99)
        LL_lisa = -0.5*np.sum(np.clip(ratio-1.0, 0, None)**2)
        # CMB prior
        LL_cmb = -0.5*((DN - DNeff_mean)/DNeff_sigma)**2
        LL = LL_pta + LL_lisa + LL_cmb
        LL_E_D[i,j] = LL
        if LL > best['LL']:
            best.update({'LL':LL,'E':E,'DNeff':DN,'n1':n1,'n2':n2,'Omega0':Omega0_guess,'f_br':fbr})

post = np.exp(LL_E_D - LL_E_D.max())
X,Y = np.meshgrid(DNeff_grid, E_grid)
plt.figure(figsize=(6.4,4.6))
plt.contourf(Y, X, post, levels=20)
levels = [np.exp(-0.5*5.99), np.exp(-0.5*2.30)]  # 95%, 68%
plt.contour(Y, X, post, levels=levels, colors=['k','k'], linestyles=['-','--'])
plt.xscale('log'); plt.xlabel('E_lambda [GeV]'); plt.ylabel('Delta N_eff'); plt.title('Posterior (toy)'); plt.tight_layout()
plt.savefig('v3_posterior_E_vs_DNeff.png', dpi=200, bbox_inches='tight'); plt.close()
print('Best-fit (toy):', best)
print('Wrote v3_posterior_E_vs_DNeff.png')
